Tiniest Blockchain

In less than 50 lines of code

https://medium.com/crypto-currently/lets-build-the-tiniest-blockchain-e70965a248b

In [6]:
import sys
sys.version_info.major

3

In [17]:
import hashlib as hasher
import sys

class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()
        
    def hash_block(self):
        # sha = hasher.sha256()
        msg = str(self.index) + \
                str(self.timestamp) + \
                str(self.data) + \
                str(self.previous_hash)
        msg = msg.encode('utf-8')
        # sha.update(hash_data)
        # return sha.hexdigest()
        if sys.version_info.major == 2:
            return unicode(hasher.sha256(msg).hexdigest(), 'utf-8')
        else:
            return hasher.sha256(str(msg).encode('utf-8')).hexdigest()

In [10]:
# genesis block with index 0, with arbitary data, and arbitrary prev hash value

import datetime as date

def create_genesis_block():
    # Manually construct a block with
    # index zero and arbitrary previous hash
    return Block(0, date.datetime.now(), "Genesis Block", "0")

In [11]:
# next block hashes info from previous block

def next_block(last_block):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = "Hey! I'm block " + str(this_index)
    this_hash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_hash)

In [18]:
# Now we can create our blockchain (simple Python list)
# First element is genesis block, then add succeeding blocks

# Create the blockchain and add the genesis block
blockchain = [create_genesis_block()]
previous_block = blockchain[0]

# How many blocks we should add to the chain
# after the genesis block
num_of_blocks_to_add = 20

# Add blocks to the chain
for i in range(0, num_of_blocks_to_add):
    block_to_add = next_block(previous_block)
    blockchain.append(block_to_add)
    previous_block = block_to_add
    # Tell everyone about it!
    print("Block #{} has been added to the blockchain!".format(block_to_add.index))
    print("Hash: {}\n".format(block_to_add.hash))

Block #1 has been added to the blockchain!
Hash: 3f84e4525393ebac01efb59d32a9f69f240f3a13e821cea4dc545248e52aa452

Block #2 has been added to the blockchain!
Hash: bd65ae3382c805b1c9fe69ad047e15becf373905f5a32a9f14407bae01083499

Block #3 has been added to the blockchain!
Hash: 04d67a4cb46efe293acddbbce6f647ff764726455cd61afa1be5d52908e0edda

Block #4 has been added to the blockchain!
Hash: d4c39793cb21bf15b069feea31513cf96227c9af12b8da7225f22fe56f43e753

Block #5 has been added to the blockchain!
Hash: 6efab82ea53984eb6c2a8b7117488c398199978bd03dfa571fa20a6306ae025d

Block #6 has been added to the blockchain!
Hash: edc6641404cf6100e90ebc2558cc8f91c0bd4e9c233ac2118e1f13f7965fb0d6

Block #7 has been added to the blockchain!
Hash: 7e82d690634be67b53078facc0def04316fbafed7a46f2d59255a8e8b2f48a97

Block #8 has been added to the blockchain!
Hash: 267835a3abc3573fb92598ecd67705b7fcbb36626c36a6bd849bbefe0d9ac4d7

Block #9 has been added to the blockchain!
Hash: 5f0fe03eceb3a6cec857511041ee601

Part 2: Tinest Blockchain Bigger

https://medium.com/crypto-currently/lets-make-the-tiniest-blockchain-bigger-ac360a328f4d

The tiniest blockchain was extremely simple, and it was relatively easy to make. But, with its simplicity came a few flaws. First, SnakeCoin only ran on one single machine, so it was far from distributed, let alone decentralized. Second, blocks could be added to the chain as fast as the host computer could create a Python object and add it to a list. In the case of a simple blockchain, that’s not a problem, but we’re now going to let SnakeCoin be an actual cryptocurrency so we’ll need control the amount of blocks (and coins) that can be created at a time.

From now on, SnakeCoin’s data will be transactions, so each block’s data field will be a list of some transactions. We’ll define a transaction as follows. Each transaction will be a JSON object detailing the sender of the coin, the receiver of the coin, and the amount of SnakeCoin that is being transferred. Note: Transactions are in JSON format for a reason I’ll detail shortly.

```json
{
  "from": "71238uqirbfh894-random-public-key-a-alkjdflakjfewn204ij",
  "to": "93j4ivnqiopvh43-random-public-key-b-qjrgvnoeirbnferinfo",
  "amount": 3
}
```

Now that we know what our transactions will look like, we need a way to add them to one of the computers in our blockchain network, called a node. To do that, we’ll create a simple HTTP server so that any user can let our nodes know that a new transaction has occurred. A node will be able to accept a POST request with a transaction (like above) as the request body. This is why transactions are JSON formatted; we need them to be transmitted to our server in a request body.

```
pip install flask
```

In [ ]:
from flask import Flask
from flask import request
node = Flask(__name__)

# Store the transaction that
# this node has in a list
this_nodes_transactions = []

@node.route('/txion', method=['POST'])
def transaction():
    if request.method == 'POST':
        # On each new post request,
        # we extract the transaction data
        new_txion = request.get_json()
        # Then we add the transaction to our list
        this_nodes_transactions.append(new_txion)
        # Because the transaction was successfully
        # submitted, we log it to our console
        print ("New transaction")
        print ("FROM: {}".format(new_txion['from']))
        print ("TO: {}".format(new_txion['to']))
        print ("AMOUNT: {}\n".format(new_txion['amount']))
        # Then we let the client know it worked out
        return "Transaction submission successful\n"

node.run()

In [ ]:
# proof of work (PoW) and /mine

miner_address = "q3nf394hjg-random-miner-address-34nf3i4nflkn3oi"

def proof_of_work(last_proof):
    # Create a variable that we will use to find
    # our next proof of work
    incrementor = last_proof + 1
    # Keep incrementing the incrementor until
    # it's equal to a number divisible by 9
    # and the proof of work of the previous
    # block in the chain
    while not (incrementor % 9 == 0 and incrementor % last_proof == 0):
        incrementor += 1
    # Once that number is found
    # we can return it as a proof
    # of our work
    return incrementor

@node.route('/mine', methods = ['GET'])
def mine():
    # Get the last proof of work
    last_block = blockchain[len(blockchain) - 1]
    last_proof = last_block.data['proof-of-work']
    # Find the proof of work for
    # the current block being mined
    # Note: The program will hang here until a new
    # proof of work is found
    proof = proof_of_work(last_proof)
    # Once we find a valid proof of work
    # we know we can mine a block so
    # we reward the minder by adding a transaction
    this_nodes_transactions.append(
        {"from": "network", "to": miner_address, "amount": 1}
    )
    # Now we can gather the data needed
    # to create the new block
    new_block_data = {
        "proof-of-work": proof,
        "transactions": list(this_nodes_transaction)
    }
    new_block_index = last_block.index + 1
    new_block_timestamp = this_timestamp = date.datetime.now()
    last_block_hash = last_block.hash
    # Empty transaction list
    this_nodes_transactions[:] = []
    # Now create the
    # new block!
    mined_block = Block(
        new_block_index,
        new_block_timestamp,
        new_block_data,
        last_block_hash
    )
    blockchain.append(mined_block)
    # Let the client know we mined a block
    return json.dumps({
      "index": new_block_index,
      "timestamp": str(new_block_timestamp),
      "data": new_block_data,
      "hash": last_block_hash
    }) + "\n"

Now, we can control the number of blocks mined in a certain time period, and we can issue new coins for people in the network to send to each other. But like we said, we’re only doing this on one computer. If blockchains are decentralized, how do we make sure that the same chain is on every node? To do this, we make each node broadcast its version of the chain to the others and allow them to receive the chains of other nodes. After that, each node has to verify the other nodes’ chains so that the every node in the network can come to a consensus of what the resulting blockchain will look like. This is called a consensus algorithm.

Our consensus algorithm will be rather simple: if a node’s chain is different from another’s (i.e. there is a conflict), then the longest chain in the network stays and all shorter chains will be deleted. If there is no conflict between the chains in our network, then we carry on.

In [ ]:
@node.route('/blocks', methods=['GET'])
def get_blocks():
    chain_to_send = blockchain
    # Convert our blocks into dictionaries
    # so we can send them as json objects later
    for block in chain_to_send:
        block_index = str(block.index)
        block_timestamp = str(block.timestamp)
        block_data = str(block.data)
        block_hash = block.hash
        block = {
            "index": block_index,
            "timestamp": block_timestamp,
            "data": block_data,
            "hash": block_hash
        }
    # Send our chain to whomever requested it
    chain_to_send = json.dumps(chain_to_send)
    return chain_to_send
    
def find_new_chains():
    # Get the blockchains of every
    # other node
    other_chains = []
    for node_url in peer_nodes:
        # Get their chains using a GET request
        block = requests.get(node_url + "/blocks").content
        # Convert the JSON object to a Python dictionary
        block = json.loads(block)
        # Add it to our list
        other_chains.append(block)
    return other_chains

def consensus():
    # Get the blocks from the other nodes
    other_chains = find_new_chains()
    # If our chain isn't longest,
    # then we store the longest chain
    longest_chain = blockchain
    for chain in other_chains:
        if len(longest_chain) < len(chain):
            longest_chain = chain
    # If the longest chain wasn't ours,
    # then we set our chain to the longest
    blockchain = longest_chain